## Demo : Monitoring vcluster using Metrics Server

### Create vcluster with real node sync enabled

##### Values.yaml preparation

In [1]:
! cat values.yaml

# Refer https://github.com/loft-sh/vcluster/blob/main/charts/k3s/values.yaml for full set of customisable options

# See https://www.vcluster.com/docs/architecture/nodes#node-syncing-modes
sync:
  nodes:
    enabled: true

##### Create vcluster with the customized values.yaml

In [2]:
! vcluster create vc1 -n host-namespace -f values.yaml --expose

[info]   Creating namespace host-namespace
[info]   execute command: helm upgrade vc1 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/915975169 --namespace host-namespace --install --repository-config='' --values /tmp/2500739412 --values values.yaml
[done] √ Successfully created virtual cluster vc1 in namespace host-namespace. 
- Use 'vcluster connect vc1 --namespace host-namespace' to access the virtual cluster
- Use `vcluster connect vc1 --namespace host-namespace -- kubectl get ns` to run a command directly within the vcluster


In [8]:
! kubectl get all -n host-namespace

NAME                                               READY   STATUS    RESTARTS   AGE
pod/coredns-5df468b6b7-sm878-x-kube-system-x-vc1   1/1     Running   0          8s
pod/vc1-0                                          2/2     Running   0          22s

NAME                                         TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)                  AGE
service/kube-dns-x-kube-system-x-vc1         ClusterIP      10.96.166.30   <none>           53/UDP,53/TCP,9153/TCP   8s
service/vc1                                  LoadBalancer   10.96.39.158   172.18.255.201   443:30775/TCP            22s
service/vc1-headless                         ClusterIP      None           <none>           443/TCP                  22s
service/vc1-node-kcd-vcluster-demo-worker2   ClusterIP      10.96.160.61   <none>           10250/TCP                8s

NAME                   READY   AGE
statefulset.apps/vc1   1/1     22s


##### Accessing Vcluster

In [9]:
! vcluster connect vc1 --namespace host-namespace

[info]   Using vcluster vc1 load balancer endpoint: 172.18.255.201
[info]   Use `vcluster connect vc1 -n host-namespace -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: ./kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig ./kubeconfig.yaml get namespaces`


### Lets create sample workloads

In [10]:
! kubectl create ns ns1 --kubeconfig ./kubeconfig.yaml
! kubectl create deployment sample -n ns1 --image=nginx --replicas 2 --kubeconfig ./kubeconfig.yaml

namespace/ns1 created
deployment.apps/sample created


##### Installing Metrics Server

In [11]:
! kubectl apply --kubeconfig ./kubeconfig.yaml -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader created
clusterrole.rbac.authorization.k8s.io/system:metrics-server created
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator created
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server created
service/metrics-server created
deployment.apps/metrics-server created
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io created


In [16]:
! kubectl get pods -n kube-system --kubeconfig ./kubeconfig.yaml

NAME                              READY   STATUS    RESTARTS   AGE
coredns-5df468b6b7-sm878          1/1     Running   0          90s
metrics-server-847dcc659d-85489   1/1     Running   0          45s


In [17]:
! kubectl top pods --all-namespaces --kubeconfig ./kubeconfig.yaml 

NAMESPACE     NAME                              CPU(cores)   MEMORY(bytes)   
kube-system   coredns-5df468b6b7-sm878          2m           15Mi            
kube-system   metrics-server-847dcc659d-85489   3m           16Mi            
ns1           sample-7f75ccc884-mj7xt           0m           6Mi             
ns1           sample-7f75ccc884-rn7n7           0m           6Mi             


### Cleanup

In [18]:
! vcluster delete vc1 -n host-namespace
! kubectl delete ns host-namespace

[info]   Delete helm chart with helm delete vc1 --namespace host-namespace --kubeconfig /tmp/3188781755 --repository-config=''
[done] √ Successfully deleted virtual cluster vc1 in namespace host-namespace
[done] √ Successfully deleted virtual cluster pvc data-vc1-0 in namespace host-namespace
namespace "host-namespace" deleted


In [19]:
! kubectl get ns

NAME                 STATUS   AGE
default              Active   19m
kube-node-lease      Active   19m
kube-public          Active   19m
kube-system          Active   19m
local-path-storage   Active   19m
metallb-system       Active   18m
